## Import dependencies

In [1]:
# Standard Library Imports
import os
import sys
import re
from subprocess import call
import json
import time
import traceback
import asyncio
import aiohttp

# Third-Party Library Imports
import numpy as np
import pandas as pd
import h5py
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import webdataset as wds
import nibabel as nib
import pickle as pkl
from einops import rearrange
import torchvision.transforms as transforms
from PIL import Image
import torch
import torchio as tio
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.preprocessing import StandardScaler

## Helper functions

In [2]:
def reshape_to_2d(tensor):
    return rearrange(tensor, 'b h w c -> (b h) (c w)')

def reshape_to_original(tensor_2d, h=64, w=64, c=48):
    return rearrange(tensor_2d, "(tr h) (c w) -> tr h w c", h=h, w=w, c=c)

def header_to_dict(header):
    readable_header = {}
    for key, value in header.items():
        readable_header[key] = value
    return readable_header

def temporal_interp1d(fmri_data, change_TR):
    original_time_points = np.arange(fmri_data.shape[0])  # Time points: 0, 1, 2, ..., T-1
    new_time_points = np.arange(0, fmri_data.shape[0], change_TR)  # New time points: 0, 2, 4, ...

    reshaped_data = fmri_data.reshape(fmri_data.shape[0], -1)  # Reshape to (T, X*Y*Z)
    interpolate = interp1d(original_time_points, reshaped_data, kind='linear', axis=0, bounds_error=False, fill_value="extrapolate")
    resampled_fmri_data = interpolate(new_time_points).reshape((len(new_time_points),) + fmri_data.shape[1:])
    return resampled_fmri_data

def list_folders(bucket, prefix='', delimiter='/'):
    folder_names = []
    continuation_token = None
    while True:
        # Include the continuation token in the request if it exists
        kwargs = {'Bucket': bucket, 'Prefix': prefix, 'Delimiter': delimiter}
        if continuation_token:
            kwargs['ContinuationToken'] = continuation_token

        response = s3.list_objects_v2(**kwargs)
        folder_names.extend([x['Prefix'].split('/')[-2] for x in response.get('CommonPrefixes', [])])

        # Check if more items are available to retrieve
        if 'NextContinuationToken' in response:
            continuation_token = response['NextContinuationToken']
        else:
            break
    return folder_names

def list_all_objects(bucket, prefix):
    continuation_token = None
    while True:
        if continuation_token:
            response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix, ContinuationToken=continuation_token)
        else:
            response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)

        for content in response.get('Contents', []):
            yield content

        continuation_token = response.get('NextContinuationToken')
        if not continuation_token:
            break

def torchio_slice(data,xslice=None,yslice=None,zslice=None):    
    if xslice is None: xslice = data.shape[1] // 2
    if yslice is None: yslice = data.shape[2] // 2
    if zslice is None: zslice = data.shape[3] // 2

    fig, axs = plt.subplots(1, 3, figsize=(5,5))

    # Plot the three different slices
    axs[0].imshow(data[0, xslice], cmap='gray')
    axs[0].axis('off')
    axs[0].set_title(f'Slice [0, {xslice}]', fontsize=8)

    axs[1].imshow(data[0, :, yslice], cmap='gray')
    axs[1].axis('off')
    axs[1].set_title(f'Slice [0, :, {yslice}]', fontsize=8)

    axs[2].imshow(data[0, :, :, zslice], cmap='gray')
    axs[2].axis('off')
    axs[2].set_title(f'Slice [0, :, :, {zslice}]', fontsize=8)
    
    plt.show()
    
def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')

## Create dir to save dataset

In [3]:
subject = "sub-01"
proj_name = "nsd_minimal"
outpath=f"/weka/proj-fmri/paulscotti/fMRI-foundation-model/dataset_creation/{proj_name}"
os.makedirs(os.path.dirname(outpath), exist_ok=True)
os.makedirs(f"{outpath}/{subject}", exist_ok=True)

## Sync metadata

```bash
aws s3 rm  s3://proj-fmri/fmri_foundation_datasets/conscioustahoe/openneuro/ --recursive

aws s3 cp s3://proj-fmri/fmri_foundation_datasets/openneuro/metadata.json s3://proj-fmri/fmri_foundation_datasets/conscioustahoe/openneuro/ --region us-west-2

aws s3 ls  s3://proj-fmri/fmri_foundation_datasets/conscioustahoe/openneuro/

aws s3 sync s3://proj-fmri/fmri_foundation_datasets/openneuro/tars s3://proj-fmri/fmri_foundation_datasets/conscioustahoe/openneuro --region us-west-2
```

## Job

In [4]:
# Create a boto3 client without signing requests
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [ ]:
f.close()

def my_split_by_node(urls):
    return urls

# prep hdf5 creation
N = 188*12*40 # number of TRs in entire NSD subj01 dataset
R = 12*40 # number of runs in entire NSD subj01 dataset

# Open an HDF5 file
f = h5py.File('subj01_rawdata.h5', 'w')

# Create datasets
global_trs_dset = f.create_dataset('global_trs', shape=(N,), dtype=np.int64)
funcs_dset = f.create_dataset('funcs', shape=(N, 64, 64, 48), dtype=np.float16)
meansds_dset = f.create_dataset('meansds', shape=(R, 2, 64, 64, 48), dtype=np.float16)

###
global_tr_cnt = 0
global_run_cnt = 0

# Set the bucket name and folder name
bucket_name = 'natural-scenes-dataset'
folder_names = list_folders(bucket_name)

tio_transforms = tio.Compose(
                (
                    tio.ToCanonical(), # make sure orientation of brains are consistent (RAS+ orientation)
                    tio.RescaleIntensity(out_min_max=(0, 1)),
                    tio.Resample(3, image_interpolation='nearest'), # rescale voxels to #mm isotropic
                    tio.CropOrPad((64, 64, 48)),
                )
            )
folder_name = "nsddata_rawdata"

print(f"Processing dataset: {folder_name}.")
all_objects = list_all_objects(bucket_name, folder_name)
for obj in all_objects:
    obj_key = obj['Key']

    if '_bold.nii.gz' in obj_key and 'nsdcore_run' in obj_key and subject in obj_key:
        print(obj_key)
        func_subj = obj_key.split('/')[1]

        filename = os.getcwd()+f'/{proj_name}/'+obj_key
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        if not os.path.exists(filename):
            s3.download_file(bucket_name, obj_key, filename)

        func_nii = nib.load(filename)

        tio_image = tio.ScalarImage(tensor=np.moveaxis(func_nii.get_fdata(),-1,0).astype(np.float32), 
                                affine=func_nii.affine, 
                                dtype=np.float32)
        out = tio_transforms(tio_image)['data']

        # zscore across the run
        out_shape = out.shape
        out = out.reshape(len(out),-1)
        scalar = StandardScaler(with_mean=True, with_std=True).fit(out)
        mean = scalar.mean_
        sd = scalar.scale_
        out = (out - mean) / sd
        mean = mean.reshape([out_shape[1],out_shape[2],out_shape[3]])
        sd = sd.reshape([out_shape[1],out_shape[2],out_shape[3]])
        meansd = np.array([mean,sd])
        out = out.reshape(out_shape)

        # create 16-bit png of mean and sd volumes
        meansd_images = reshape_to_2d(meansd)
        meansd_images = torch.Tensor(meansd_images)
        min_meansd, max_meansd = meansd_images.min(), meansd_images.max()
        minmax_meansd_images = (meansd_images - min_meansd) / (max_meansd - min_meansd) # first you need to rescale to 0 to 1
        # rescaled_images = (minmax_meansd_images * 65535).to(torch.int16) # then multiply by constant prior to numpy uint16
        # rescaled_images_numpy = rescaled_images.numpy().astype(np.uint16)
        # meansd_PIL_image = Image.fromarray(rescaled_images_numpy, mode='I;16')
        
        meansds_dset[global_run_cnt] = reshape_to_original(minmax_meansd_images)
        global_run_cnt += 1

        for batch in range(len(out)):
            images = reshape_to_2d(out[[batch]])
            images = torch.Tensor(images)

            # convert tensor to something compatible with 16-bit png
            min_, max_ = images.min(), images.max()
            minmax_images = (images - min_) / (max_ - min_) # first you need to rescale to 0 to 1
            # rescaled_images = (minmax_images * 65535).to(torch.int16) # then multiply by constant prior to numpy uint16
            # rescaled_images_numpy = rescaled_images.numpy().astype(np.uint16)
            # PIL_image = Image.fromarray(rescaled_images_numpy, mode='I;16')
            
            funcs_dset[global_tr_cnt] = reshape_to_original(minmax_images)

            global_trs_dset[global_tr_cnt] = global_tr_cnt
            global_tr_cnt += 1
        print(global_run_cnt, global_tr_cnt)
###
    
f.close()

Processing dataset: nsddata_rawdata.
nsddata_rawdata/sub-01/ses-nsd01/func/sub-01_ses-nsd01_task-nsdcore_run-01_bold.nii.gz
1 188
nsddata_rawdata/sub-01/ses-nsd01/func/sub-01_ses-nsd01_task-nsdcore_run-02_bold.nii.gz
2 376
nsddata_rawdata/sub-01/ses-nsd01/func/sub-01_ses-nsd01_task-nsdcore_run-03_bold.nii.gz
3 564
nsddata_rawdata/sub-01/ses-nsd01/func/sub-01_ses-nsd01_task-nsdcore_run-04_bold.nii.gz
4 752
nsddata_rawdata/sub-01/ses-nsd01/func/sub-01_ses-nsd01_task-nsdcore_run-05_bold.nii.gz
5 940
nsddata_rawdata/sub-01/ses-nsd01/func/sub-01_ses-nsd01_task-nsdcore_run-06_bold.nii.gz
6 1128
nsddata_rawdata/sub-01/ses-nsd01/func/sub-01_ses-nsd01_task-nsdcore_run-07_bold.nii.gz
7 1316
nsddata_rawdata/sub-01/ses-nsd01/func/sub-01_ses-nsd01_task-nsdcore_run-08_bold.nii.gz
8 1504
nsddata_rawdata/sub-01/ses-nsd01/func/sub-01_ses-nsd01_task-nsdcore_run-09_bold.nii.gz
9 1692
nsddata_rawdata/sub-01/ses-nsd01/func/sub-01_ses-nsd01_task-nsdcore_run-10_bold.nii.gz
10 1880
nsddata_rawdata/sub-01/ses-

In [40]:
print(global_run_cnt, global_tr_cnt)

480 90240


# Matrix of behavioral/stimuli info

In [5]:
TR = 1.6
TRs_per_run = 188
print("TR =",TR)

shared1000 = np.load('/weka/proj-fmri/shared/mindeyev2_dataset/shared1000.npy')

TR = 1.6


In [6]:
# Set the bucket name and folder name
bucket_name = 'natural-scenes-dataset'
folder_names = list_folders(bucket_name)

folder_name = "nsddata_rawdata"
print(f"Processing dataset: {folder_name}.")

stim = {}
run_cnt = 0
last_trial = 0

pattern = r"ses-nsd(\d+)/.*?run-(\d+)"

# List all objects in the folder
all_objects = list_all_objects(bucket_name, folder_name)
for obj in all_objects:
    obj_key = obj['Key']

    if '_events.tsv' in obj_key and 'nsdcore_run' in obj_key and subject in obj_key:
        # print(obj_key)
        
        filename = os.getcwd()+f'/{proj_name}/'+obj_key
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        if not os.path.exists(filename):
            s3.download_file(bucket_name, obj_key, filename)

        events = pd.read_csv(filename, delimiter='\t')
        
        match = re.search(pattern, filename)
        ses, run = match.groups()
        
        if stim=={}:
            stim['run_TR_onsets'] = events['onset'].values / TR
            stim['global_TR_onsets'] = (events['onset'].values / TR) + (run_cnt*188)
            stim['coco73k'] = events['73k_id'].values - 1
            stim['run_trial'] = events['trial_number'].values
            stim['global_trial'] = events['trial_number'].values + last_trial
            stim['global_sess'] = np.repeat(int(ses), len(events['73k_id'].values))
            stim['global_run'] = np.repeat(int(run) + (run_cnt*12), len(events['73k_id'].values))
            stim['trial_type'] = events['trial_type'].values
        else:
            stim['run_TR_onsets'] = np.append(stim['run_TR_onsets'], events['onset'].values / TR)
            stim['global_TR_onsets'] = np.append(stim['global_TR_onsets'],(events['onset'].values / TR) + (run_cnt*188))
            stim['coco73k'] = np.append(stim['coco73k'], events['73k_id'].values - 1)
            stim['run_trial'] = np.append(stim['run_trial'], events['trial_number'].values)
            stim['global_trial'] = np.append(stim['global_trial'], events['trial_number'].values + last_trial)
            stim['global_sess'] = np.append(stim['global_sess'], np.repeat(int(ses), len(events['73k_id'].values)))
            stim['global_run'] = np.append(stim['global_run'], np.repeat(int(run) + (run_cnt*12), len(events['73k_id'].values)))
            stim['trial_type'] = np.append(stim['trial_type'], events['trial_type'].values)
            
        last_trial = stim['global_trial'][-1]
        run_cnt += 1

stim['shared1000'] = shared1000[stim['coco73k']]
        
# convert to 0-index
stim['run_trial'] = stim['run_trial']-1
stim['global_trial'] = stim['global_trial']-1
stim['global_run'] = stim['global_run']-1
stim['global_sess'] = stim['global_sess']-1

df = pd.DataFrame(stim)
df.to_csv(f'{proj_name}/{subject}/nsddata_rawdata.csv', index=False)
df.to_csv(f'nsddata_rawdata.csv', index=False)
df

Processing dataset: nsddata_rawdata.


,run_TR_onsets,global_TR_onsets,coco73k,run_trial,global_trial,global_sess,global_run,trial_type,shared1000
0,7.5,7.5,46002,0,0,0,0,0,True
1,10.0,10.0,61882,1,1,0,0,0,False
2,12.5,12.5,828,2,2,0,0,0,False
3,15.0,15.0,67573,3,3,0,0,0,False
4,17.5,17.5,16020,4,4,0,0,0,False
...,...,...,...,...,...,...,...,...,...
29995,162.5,90214.5,13773,57,29995,39,5759,1,False
29996,165.0,90217.0,66767,58,29996,39,5759,1,False
29997,167.5,90219.5,53167,59,29997,39,5759,1,False
29998,170.0,90222.0,1943,60,29998,39,5759,1,False


# Send to s3

In [30]:
# send to aws s3
command = f"aws s3 sync {outpath}/{subject} s3://proj-fmri/fmri_foundation_datasets/{proj_name} --region us-west-2"
call(command,shell=True)

# # delete tars
# command = f"rm {outpath}/{subject}/*.tar"
# call(command,shell=True)

upload: nsd_minimal/sub-01/nsddata_rawdata.csv to s3://proj-fmri/fmri_foundation_datasets/nsd_minimal/nsddata_rawdata.csv


0

# Making tars

In [ ]:
global_tr_cnt = 0
TRs_per_sample = 24
max_samples_per_tar = 320 # translates to around 1 Gb per tar
max_TRs_per_tar = max_samples_per_tar * TRs_per_sample

# Set the bucket name and folder name
bucket_name = 'natural-scenes-dataset'
folder_names = list_folders(bucket_name)

print(f"TRs_per_sample: {TRs_per_sample} max_samples_per_tar: {max_samples_per_tar} max_TRs_per_tar: {max_TRs_per_tar}")

# If you want to start over webdataset creation from the beginning, uncomment these:
tar_count = 0
TR_count = 0
subj_count = 0
dataset_count = 0
dataset_list = []
obj_key_list = []

sample_idx = 0
current_dataset = None
current_subject = None
sink = wds.TarWriter(f"{outpath}/sub-01/{tar_count:06d}.tar")

tio_transforms = tio.Compose(
                (
                    tio.ToCanonical(), # make sure orientation of brains are consistent (RAS+ orientation)
                    tio.RescaleIntensity(out_min_max=(0, 1)),
                    tio.Resample(3, image_interpolation='nearest'), # rescale voxels to #mm isotropic
                    tio.CropOrPad((64, 64, 48)),
                )
            )

folder_name = "nsddata_rawdata"

print(f"Processing dataset: {folder_name}.")
all_objects = list_all_objects(bucket_name, folder_name)
for obj in all_objects:
    obj_key = obj['Key']

    if '_bold.nii.gz' in obj_key and 'nsdcore_run' in obj_key and subject in obj_key:
        print(obj_key)
        func_subj = obj_key.split('/')[1]

        # if metadata file shows you already processed this file, skip it
        if np.isin(obj_key, obj_key_list):
            continue

        filename = os.getcwd()+f'/{proj_name}/'+obj_key
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        if not os.path.exists(filename):
            s3.download_file(bucket_name, obj_key, filename)

        func_nii = nib.load(filename)
        try:
            print(obj_key, func_nii.get_fdata().shape, "| TRs:", TR_count, "samp:", sample_idx)
        except Exception as e:
            print(f"get_fdata() error occurred: {e}")
            continue

        try:
            tio_image = tio.ScalarImage(tensor=np.moveaxis(func_nii.get_fdata(),-1,0).astype(np.float32), 
                                    affine=func_nii.affine, 
                                    dtype=np.float32)
            out = tio_transforms(tio_image)['data']
        except Exception as e: # this can happen if the func is actually 3d and not 4d
            print(f"tio processing error occurred: {e}")
            continue

        # zscore across the run
        out_shape = out.shape
        out = out.reshape(len(out),-1)
        scalar = StandardScaler(with_mean=True, with_std=True).fit(out)
        mean = scalar.mean_
        sd = scalar.scale_
        out = (out - mean) / sd
        mean = mean.reshape([out_shape[1],out_shape[2],out_shape[3]])
        sd = sd.reshape([out_shape[1],out_shape[2],out_shape[3]])
        meansd = np.array([mean,sd])
        out = out.reshape(out_shape)

        # create 16-bit png of mean and sd volumes
        meansd_images = reshape_to_2d(meansd)
        meansd_images = torch.Tensor(meansd_images)
        min_meansd, max_meansd = meansd_images.min(), meansd_images.max()
        minmax_meansd_images = (meansd_images - min_meansd) / (max_meansd - min_meansd) # first you need to rescale to 0 to 1
        rescaled_images = (minmax_meansd_images * 65535).to(torch.int16) # then multiply by constant prior to numpy uint16
        rescaled_images_numpy = rescaled_images.numpy().astype(np.uint16)
        meansd_PIL_image = Image.fromarray(rescaled_images_numpy, mode='I;16')
        
        global_tr_cnt += len(out)

        # create samples of TRs_per_sample TRs
        for batch in range(0,len(out),TRs_per_sample):
            if len(out[batch:batch+TRs_per_sample])<TRs_per_sample:
                continue
            images = reshape_to_2d(out[batch:batch+TRs_per_sample])
            images = torch.Tensor(images)

            # convert tensor to something compatible with 16-bit png
            min_, max_ = images.min(), images.max()
            minmax_images = (images - min_) / (max_ - min_) # first you need to rescale to 0 to 1
            rescaled_images = (minmax_images * 65535).to(torch.int16) # then multiply by constant prior to numpy uint16
            rescaled_images_numpy = rescaled_images.numpy().astype(np.uint16)
            PIL_image = Image.fromarray(rescaled_images_numpy, mode='I;16')

            sink.write({
                "__key__": "%06d" % sample_idx,
                "dataset.txt": obj_key,
                "header.npy": np.array(header_to_dict(func_nii.header)),
                "minmax.npy": np.array([min_, max_, min_meansd, max_meansd]),
                "meansd.png": meansd_PIL_image,
                "func.png": PIL_image, # 27M for 8-bit png vs 48M for 16-bit png vs 144M for numpy
                "run_TR.npy": np.arange(len(out))[batch:batch+TRs_per_sample],
                "global_TR.npy": np.arange(len(out))[batch:batch+TRs_per_sample] + (global_tr_cnt - len(out)),
            })

            if current_dataset != obj['Key'].split('/')[0]:
                print(obj_key, "| TR_count:", TR_count, "sample_idx:", sample_idx)
                dataset_list.append(obj['Key'].split('/')[0])
                dataset_count += 1
                if is_interactive(): # dont want to plot unless you are in interactive notebook
                    torchio_slice(out) # plot normalized slices
                    torchio_slice((out * sd) + mean) # plot unnormalized slices

            if current_subject != obj['Key'].split('/')[1]:
                subj_count += 1
            current_dataset = obj['Key'].split('/')[0]
            current_subject = obj['Key'].split('/')[1]

            TR_count += TRs_per_sample
            sample_idx += 1

            if sample_idx >= max_samples_per_tar:
                print("HIT MAX SAMPLES PER TAR")
                sink.close()
                sample_idx = 0 

                # make metadata file and save progress to aws s3
                data = {
                    "TR_count": TR_count,
                    "subj_count": subj_count,
                    "tar_count": tar_count,
                    "dataset_count": dataset_count,
                    "datasets": dataset_list,
                    "obj_key_list": obj_key_list,
                }
                with open(f"{outpath}/{subject}/metadata.json", "w") as file:
                    json.dump(data, file)

                tar_count += 1
                sink = wds.TarWriter(f"{outpath}/{subject}/{tar_count:06d}.tar")

        obj_key_list.append(obj['Key'])

print("TR_count",TR_count)
print("subj_count",subj_count)
print("dataset_count",dataset_count)
print("tar_count", tar_count)   

try:
    sink.close()
except:
    pass

data = {
    "TR_count": TR_count,
    "subj_count": subj_count,
    "tar_count": tar_count,
    "datasets": dataset_list,
    "obj_key_list": obj_key_list,          
}

with open(f"{outpath}/{subject}/metadata.json", "w") as file:
    json.dump(data, file)